In [1]:
%load_ext autoreload
%autoreload 2

import os
os.chdir("../")

%pylab inline
pylab.rcParams['figure.figsize'] = (10, 5)

Populating the interactive namespace from numpy and matplotlib


In [2]:
import torch.backends.cudnn as cudnn
import torch
import random
import ld_gan
import numpy as np

RAND_SEED = 42
cudnn.benchmark = True
random.seed(RAND_SEED)
torch.manual_seed(RAND_SEED)
torch.cuda.manual_seed_all(RAND_SEED)


BATCH_SIZE  = 256
LATENT_SIZE = 256
LR_GAN = 0.0002
LR_AE  = 0.0001
LR_CLF = 0.001


X, Y, Xt, Yt = ld_gan.data_proc.data_loader.load_data(-1, 
                                                      verbose=1, 
                                                      resize = 64,
                                                      split_test_train_ratio = 0.2)
n_classes = Y.shape[1]
Y = np.argmax(Y, axis = 1)
Yt = np.argmax(Yt, axis = 1)


gen = ld_gan.models.gen.gen_64(latent_size = LATENT_SIZE)
dis = ld_gan.models.dis.dis_64()
enc = ld_gan.models.enc.Enc(n_features = LATENT_SIZE, activation = None)
enc.cuda()
enc.apply(ld_gan.models.init_weights)

train_ops = [
                #ld_gan.train_ops.GanDis(gen, dis, LR_GAN),
                #ld_gan.train_ops.GanGen(gen, dis, LR_GAN),
                ld_gan.train_ops.Clf(enc, LR_CLF, LATENT_SIZE, n_classes)
            ]


100%|██████████| 102/102 [00:11<00:00, 12.36it/s]

load data from '/export/home/oblum/projects/ls_gan/data/flowers_102/jpg_128'


ValueError: optimizer got an empty parameter list

In [2]:
import ld_gan
import torch.nn as nn
import numpy as np

In [3]:
X, Y, Xt, Yt = ld_gan.data_proc.data_loader.load_data(1, 
                                                      verbose=1, 
                                                      resize = 64,
                                                      split_test_train_ratio = 0.2)
n_classes = Y.shape[1]
Y = np.argmax(Y, axis = 1)
Yt = np.argmax(Yt, axis = 1)

100%|██████████| 102/102 [00:12<00:00, 11.21it/s]

load data from '/export/home/oblum/projects/ls_gan/data/flowers_102/jpg_128'


In [4]:
from ld_gan.models import init_weights

class Enc(nn.Module):
    
    def __init__(self, 
                 ipt_size   = 64,
                 complexity = 64,
                 n_col      = 3,
                 n_features = 256,
                 activation = "tanh", 
                 add_clf_layer = False, 
                 n_classes = 102):
        
        super(Enc, self).__init__()
        
        self.activation = activation
        self.add_clf_layer = add_clf_layer
        self.n_blocks = int(np.log2(64) - 1)
        
        self.main = nn.Sequential()
        
        # BLOCK 0
        self.main.add_module('b00', nn.Conv2d(n_col, complexity, 4, 2, 1, bias=False))
        self.main.add_module('b01', nn.LeakyReLU(0.2, inplace=True))

        # BLOCKS 1 - N-1
        for b in range(1, self.n_blocks - 1):
            c_in  = complexity * 2**(b-1)
            c_out = complexity * 2**(b)
            n = 'b' + str(b)
            self.main.add_module(n+'0', nn.Conv2d(c_in, c_out, 4, 2, 1, bias=False))
            self.main.add_module(n+'1', nn.BatchNorm2d(c_out))
            self.main.add_module(n+'2', nn.LeakyReLU(0.2, inplace=True))
        
        # BLOCK N: 4 --> 1
        n = 'b' + str(self.n_blocks-1) + "0"
        self.main.add_module(n, nn.Conv2d(c_out, n_features, 4, 1, 0, bias=False))
        
        # CLF-LAYER
        if self.add_clf_layer:
            self.clf = nn.Linear(n_features, n_classes)

    def forward(self, x):
        
        x = self.main(x)
            
        if self.activation == "tanh":
            x = F.tanh(x)
            
        if self.add_clf_layer:
            x = x.view(x.size(0), x.size(1))
            x = self.clf(x)
            #x = F.softmax(x)
        
        return x

In [5]:
enc = Enc()

In [6]:
enc.cuda()
enc.apply(ld_gan.models.init_weights)

Enc (
  (main): Sequential (
    (b00): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (b01): LeakyReLU (0.2, inplace)
    (b10): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (b11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
    (b12): LeakyReLU (0.2, inplace)
    (b20): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (b21): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
    (b22): LeakyReLU (0.2, inplace)
    (b30): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (b31): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True)
    (b32): LeakyReLU (0.2, inplace)
    (b40): Conv2d(512, 256, kernel_size=(4, 4), stride=(1, 1), bias=False)
  )
  (clf): Linear (256 -> 102)
)

In [7]:
sampler = ld_gan.sample.generate_rand_noise(X, Y, 256, 256)

In [8]:
criterion = nn.CrossEntropyLoss()

In [9]:
from ld_gan.data_proc.transformer import np_to_tensor, tensor_to_np
import torch.optim as optim
import torch.nn.functional as F
import torch

In [10]:
opt = optim.Adam(enc.parameters(), lr=0.001)

In [12]:
errs = []

for i in range(1000):
    
    X, Y, Z, Z_bar = sampler.next()
    X, Y, Z, Z_bar = np_to_tensor(X, Y, Z, Z_bar)
    
    enc.zero_grad()
    y = enc(X)
    err = criterion(y, Y)
    err.backward()
    mean = y.data.mean()

    errs.append(err.cpu().data.numpy()[0])
    
    opt.step()
    
    if i % 100 == 0:
        yt = Yt[:256]
        y = enc(ld_gan.data_proc.transformer.np_to_tensor(Xt[:256]))
        y = ld_gan.data_proc.transformer.tensor_to_np(y)
        y = np.argmax(y, axis = 1)
        acc = float((yt == y).sum()) / len(yt)
        print "\n"
        print "accuracy: ", acc
        print "loss:", np.array(errs).mean()
        errs = []



accuracy:  0.0625
loss: 4.40144


accuracy:  0.33984375
loss: 2.85177


accuracy:  0.42578125
loss: 1.5057


accuracy:  0.4375
loss: 0.589584


accuracy:  0.4921875
loss: 0.135699


accuracy:  0.50390625
loss: 0.0349066


accuracy:  0.49609375
loss: 0.0171484


accuracy:  0.5078125
loss: 0.0114115


accuracy:  0.515625
loss: 0.00845197


accuracy:  0.50390625
loss: 0.00657259


In [63]:
y = enc(ld_gan.data_proc.transformer.np_to_tensor(Xt[:256]))

In [64]:
y.size()

torch.Size([256, 102])

In [65]:
y = ld_gan.data_proc.transformer.tensor_to_np(y)

In [66]:
y = np.argmax(y, axis = 1)

In [67]:
y

array([  1,  43,  58,  33,  12,  45,  50,  16,  64,   5,  82,  33,  84,
         1,   6,  25,  61,  28,  57,  60,   4,   3,  84,  77,  26,  72,
        69,  90,  29,  70,   9,  91, 101,  10,  33,  13,  43,  22,  97,
        14,  23,  18,  12,  50,  29,   3,  66,  93,  43,  82,  26,  10,
        19,  33,  37,   3,  96,  74,  98,  24,  71,  95,  13,  33,  93,
        12,   7,   1,  31,   5,  26,  25,   1,  93,  30,  40,   0,  13,
        37,  90,   3,  26,  97,  41,  93,  39,  11,   4,  38,  32,  11,
        49,  39,  16,  12,   5,  54,  26,  25,  13,  62,  48,  42,  16,
        27,  27,   1,  42,  53,  16,  94,  49,  19,  23,  50,  23,   7,
         3,  93,  11, 101,  29,  69,  33,   7,  95,  53,  25,   9,   5,
         9,  37,  11,  82,  56,   8,  88,   7, 100,   0,  24,  28,  23,
         1,  15,  24,  56,  15,  18,  74,  54,  73,  15,  98,   7,   3,
        29,  61,  36,  12,  45,  14,  28,  93,  37,  97,   4,   7,  93,
        29,  22,  80,  41,   9,  11,  69,  14,  91,  93,  10,  9

In [68]:
yt = Yt[:256]

In [69]:
acc = float((y == yt).sum()) / len(yt)

In [70]:
acc

0.328125

In [29]:
import torch.optim as optim
from torch import nn
import torch
from torch.autograd import Variable
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):

        super(Net, self).__init__()
        self.l1 = nn.Linear(10, 84)
        self.l2 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        return x

net = Net()

# define 
criterion = nn.L1Loss()
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)

# define random batch
dtype = torch.FloatTensor
x = torch.randn(100, 10).type(dtype)
y = torch.randn(100, 10).type(dtype)
x, y = Variable(x), Variable(y)

# train with one batch
optimizer.zero_grad()
y_pred = net(x)
loss = criterion(y_pred, y)
loss.backward()
optimizer.step()

ValueError: optimizer got an empty parameter list

In [4]:
import torch.optim as optim
from torch import nn
import torch
from torch.autograd import Variable
import torch.nn.functional as F

# define network
class Net(nn.Module):
    def __init__(self):

        super(Net, self).__init__()
        self.main = nn.Sequential(
            nn.Linear(10, 84),
            nn.Linear(84, 10)
        )

    def forward(self, input):
        output = self.main(input)
        return output

net = Net()

# define loss and optimisation algorithm
criterion = nn.L1Loss()
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)

# define random batch
dtype = torch.FloatTensor
x = torch.randn(100, 10).type(dtype)
y = torch.randn(100, 10).type(dtype)
x, y = Variable(x), Variable(y)

# train with one batch
net.zero_grad()
y_pred = net(x)
loss = criterion(y_pred, y)
loss.backward()
optimizer.step()

In [4]:
ld_gan.models.gen.gen_64()

gen_64 (
  (main): Sequential (
    (0): ConvTranspose2d(100, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True)
    (2): ReLU (inplace)
    (3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
    (5): ReLU (inplace)
    (6): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
    (8): ReLU (inplace)
    (9): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
    (11): ReLU (inplace)
    (12): ConvTranspose2d(64, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (13): Tanh ()
  )
)